In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from pyproj import CRS
from itertools import chain
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

## Load grid data

In [ ]:
data_file = 'sardinia_geo.pkl'
force = False

if force or not os.path.isfile(data_file):
    powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2023 SP2\Python\3.10'
    if powerfactory_path not in sys.path:
        sys.path.append(powerfactory_path)
    import powerfactory as pf

    app = pf.GetApplication()
    if app is None:
        raise Exception('Cannot get PowerFactory application')
    print('Got PowerFactory application.')
    
    project_name = '\\Terna_Inerzia\\V2020_Rete_Sardegna_2021_06_03cr'
    err = app.ActivateProject(project_name)
    if err:
        raise Exception(f'Cannot activate project {project_name}')
    print(f'Activated project "{project_name}".')
    
    def get_coords(obj_type):
        objs = app.GetCalcRelevantObjects('*.' + obj_type)
        names, coords = [], []
        for obj in objs:
            if (obj.HasAttribute('outserv') and obj.outserv) or \
            not obj.HasAttribute('GPSlat') or not obj.HasAttribute('GPSlon'):
                continue
            lat,long = obj.GPSlat, obj.GPSlon
            if lat != 0.0 and long != 0.0:
                coords.append([lat,long])
                names.append(obj.loc_name)
        return names, np.array(coords)

    obj_types = {'sites': 'ElmSite', 'terminals': 'ElmTerm', 'substations': 'ElmSubstat'}
    names, coords = {}, {}
    for k,v in obj_types.items():
        names[k], coords[k] = get_coords(v)
    lines = [obj for obj in app.GetCalcRelevantObjects('*.ElmLne') if not obj.outserv]
    names['lines'], coords['lines'], line_ratings = [], [], []
    for line in lines:
        if len(line.GPScoords) > 0:
            names['lines'].append(line.loc_name)
            coords['lines'].append(np.array(line.GPScoords))
            line_ratings.append(line.typ_id.uline)
    coords['lines'] = np.array(coords['lines'], dtype=object)
    line_ratings = np.array(line_ratings)

    data = {
        'names': names,
        'coords': coords,
        'line_ratings': line_ratings
    }
    pickle.dump(data, open(data_file, 'wb'))
    
else:
    data = pickle.load(open(data_file, 'rb'))
    names = data['names']
    coords = data['coords']
    line_ratings = data['line_ratings']

## Coordinate reference systems

In [ ]:
# 3035: Lambert azimuthal equal area
# 3857: spherical Mercator projection
# 4326: world geodetic system 1984
coord_ref = 3857

Convert the coordinates of the grid to the chosen reference system:

In [ ]:
gdf = {key: gpd.GeoDataFrame(data={'name': names[key], 'geometry':
                             [Point(long,lat) for lat,long in
                              zip(coords[key][:,0], coords[key][:,1])]})
       for key in ('sites','substations','terminals')}
gdf['lines'] = gpd.GeoDataFrame(data={'name': names['lines'], 'vrating': line_ratings,
                                      'geometry': [LineString(np.array([line[:,1], line[:,0]]).T) 
                                                   for line in coords['lines']]})
vratings = np.unique(gdf['lines']['vrating'])[::-1]

for key in gdf:
    gdf[key].crs = CRS.from_user_input(4326)
    gdf[key] = gdf[key].to_crs(epsg=coord_ref)
    gdf[key] = gdf[key].sort_values('name').set_index('name')

In [ ]:
gdf['lines']

## Map of Europe

First define the bounding box:

In [ ]:
limits = {'WS': Point(8, 38.75), 'EN': Point(10, 41.4)}
bbox = gpd.GeoDataFrame(data=limits.values(),
                        index=pd.Index(data=limits.keys(), name='name'),
                        columns=['geometry'])
bbox.crs = CRS.from_user_input(4326)
bbox = bbox.to_crs(epsg=coord_ref)

Then load the data and keep only those coordinates that fall within the bounding box:

In [ ]:
scale = 1 # 1 : 1,000,000
year = 2021
europe_folder = f'geography/ref-nuts-{year}-{scale:02d}m'
N_levels = 4
map_types = 'BN', #'LB' # BN: boundary, LB: label, RG: region
europe = {map_type: {} for map_type in map_types}
for level in range(N_levels):
    for map_type in map_types:
        if map_type == 'LB':
            europe_file = f'{europe_folder}/NUTS_{map_type}_{year}_{coord_ref}_LEVL_{level}.json'
        else:
            europe_file = f'{europe_folder}/NUTS_{map_type}_{scale:02d}M_{year}_{coord_ref}_LEVL_{level}.json'
        tmp = gpd.read_file(europe_file)
        tmp.crs = CRS.from_user_input(coord_ref)
        europe[map_type][level] = tmp.cx[bbox.loc['WS','geometry'].x : bbox.loc['EN','geometry'].x,
                                         bbox.loc['WS','geometry'].y : bbox.loc['EN','geometry'].y]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(4,8))
light_gray = .8 + np.zeros(3)
dark_gray = .2 + np.zeros(3)
europe['BN'][0].plot(ax=ax, lw=1.25, color=dark_gray)
europe['BN'][3].plot(ax=ax, lw=0.5, color=light_gray)
for vrat in vratings:
    idx = gdf['lines']['vrating'] == vrat
    gdf['lines'].loc[idx,:].plot(ax=ax, lw=np.floor(vrat/100), color='tab:blue', label=f'{vrat:.0f} kV')
msizes = {'sites': 64, 'substations': 16, 'terminals': 4}
cmap = {'sites': [.5,1,.2], 'substations': [1,.25,1], 'terminals': [0,0,0]}
for type_name in cmap:
    gdf[type_name].plot(marker='o', ax=ax, markersize=msizes[type_name],
                        color=cmap[type_name], label=type_name)
ax.legend(loc='upper left', fontsize=8, frameon=False)
ax.axis('off')
fig.tight_layout()